In [49]:
from matplotlib import pyplot as plt
import numpy as np
from scipy.optimize import curve_fit as cf
%matplotlib

data = np.loadtxt("6mt9.pdb.dat",
                  dtype={'names': ('q', 'I(q)', 'I(q)_Error'),'formats': (np.float, np.float, np.float)}, skiprows=2)



Using matplotlib backend: MacOSX


In [50]:
'''Before doing any analysis, we need to define a few functions
Start with a basic function that we will pass into a NLS algorithm
& write our own function that takes into consideration signal error
'''

def lineModel(x,m,b):
    return b*x+m

def lsqfity_with_sigmas(X, Y, SIG):
    """ This is a special version of linear least squares that uses
    known sigma values of Y to calculate standard error in slope and
    intercept. The actual fit is also SIGMA-weighted
    """
    # X, Y, SIG = map(np.asanyarray, (X, Y, SIG))
    # print "SIG: ", SIG
    # print "I: ", Y
    XSIG2 = np.sum(X / (SIG ** 2))
    X2SIG2 = np.sum((X ** 2) / (SIG ** 2))
    INVSIG2 = np.sum(1.0 / (SIG ** 2))
    XYSIG2 = np.sum(X * Y / (SIG ** 2))
    YSIG2 = np.sum(Y / (SIG ** 2))
    delta = INVSIG2 * X2SIG2 - XSIG2 ** 2
    # print ":::: ", XSIG2, X2SIG2, INVSIG2, XYSIG2, YSIG2, "delta: ", delta
    smy = X2SIG2 / delta
    sby = INVSIG2 / delta
    my = (X2SIG2 * YSIG2 - XSIG2 * XYSIG2) / delta
    by = (INVSIG2 * XYSIG2 - XSIG2 * YSIG2) / delta
    # ouput intercept(my), slope(by), sigma intercept(smy), sigma slope(sby)
    return my, by, smy, sby




In [51]:
'''Now we have a way to perform Guiner analysis
1st attempt will use scipy
2nd attempt will use our lsqfity_with_sigmas function
& we will compare the results
'''
nmin=0
nmax=120

guess=[12,-5]
c,cov = cf(lineModel, data['q'][nmin:nmax]**2, np.log(data['I(q)'][nmin:nmax]),guess,sigma=data['I(q)_Error'][nmin:nmax]/data['I(q)'][nmin:nmax])
print(c)

inter,slope,siginter,slopeint = lsqfity_with_sigmas(data['q'][nmin:nmax]**2,
                                                    np.log(data['I(q)'][nmin:nmax]),np.log(data['I(q)_Error'][nmin:nmax]))
print(inter,slope)

plt.plot(data['q'][nmin:nmax]**2, np.log(data['I(q)'][nmin:nmax]))
plt.plot(data['q'][nmin:nmax]**2, lineModel(data['q'][nmin:nmax]**2,inter,slope))
plt.show()

'''Extract and print out the Rg value from both determinations'''

print('The Rg(A) determined from scipy fit: %.2f'% np.sqrt(-3*slope))
print('The Rg(A) determined from our fit: %.2f'%np.sqrt(-3*c[1]))

[  18.2718218  -243.05713112]
18.277101673064315 -244.329220304641
The Rg(A) determined from scipy fit: 27.07
The Rg(A) determined from $our$ fit: 27.00
